In [30]:
%pip install yfinance
%pip install scipy
%pip install pandas
%pip install numpy
%pip install statsmodels
%pip install pingouin

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/198.6 kB ? eta -:--:--
     -------------------------------------  194.6/198.6 kB 5.9 MB/s eta 0:00:01
     -------------------------------------- 198.6/198.6 kB 6.1 MB/s eta 0:00:00
  Obtaining dependency information for pandas-flavor>=0.2.0 from https://files.pythonhosted.org/packages/67/1a/bfb5574b215f530c7ac5be684f33d60b299abbebd763c203aa31755f2fb2/pandas_flavor-0.6.0-py3-none-any.whl.metadata
  Obtaining dependency information for xarray from https://files.pythonhosted.org/packages/cc/21/c3a9d3017c131cea3c2967957b05448ffc81cafced239796bbbdc98073e4/xarray-2023.8.0-py3-none-any.whl.metadata
  Installing build 

In [36]:
import pandas as pd
import numpy as np
from scipy import stats
import yfinance as yf
import plotly.graph_objects as go
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import pingouin as pg
from pingouin import mixed_anova, read_dataset
from statsmodels.stats.anova import AnovaRM

In [3]:
ibov_ohlc = yf.download('^BVSP', period='max', interval='1d')

ibov_ohlc.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-04-27,24.799999,25.400000,24.500000,24.500000,24.500000,0
1993-04-28,24.500000,24.600000,23.700001,24.299999,24.299999,0
1993-04-29,24.299999,24.799999,23.700001,23.700001,23.700001,0
1993-04-30,23.700001,24.200001,23.700001,24.100000,24.100000,0
1993-05-03,24.100000,24.400000,23.799999,24.100000,24.100000,0


In [4]:
ibov_ohlc = ibov_ohlc.resample('M').agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})

In [5]:
ibov_ohlc.head()

,Open,High,Low,Close
Date,,,,
1993-04-30,24.799999,25.400000,23.700001,24.100000
1993-05-31,24.100000,35.599998,23.799999,34.900002
1993-06-30,35.400002,54.799999,35.400002,53.700001
1993-07-31,53.700001,67.400002,51.700001,65.599998
1993-08-31,65.599998,100.099998,60.599998,96.400002


In [6]:
ibov_ohlc['Resultado_mes'] = (ibov_ohlc['Close'].pct_change()*100).round(2)

In [7]:
ibov_ohlc.head()

,Open,High,Low,Close,Resultado_mes
Date,,,,,
1993-04-30,24.799999,25.400000,23.700001,24.100000,NaN
1993-05-31,24.100000,35.599998,23.799999,34.900002,44.81
1993-06-30,35.400002,54.799999,35.400002,53.700001,53.87
1993-07-31,53.700001,67.400002,51.700001,65.599998,22.16
1993-08-31,65.599998,100.099998,60.599998,96.400002,46.95


In [17]:

ibov_ohlc["Mes"] = ibov_ohlc.index.month_name()
ibov_ohlc["Ano"] = ibov_ohlc.index.year
ibov_ohlc["Mes_num"] = ibov_ohlc.index.strftime('%m')
ibov_ohlc = ibov_ohlc.loc['2000':'2023']
ibov_ohlc

,Open,High,Low,Close,Resultado_mes,Mes,Ano,Mes_num
Date,,,,,,,,
2000-01-31,17098.0,18099.0,15350.0,16388.0,-4.12,January,2000,01
2000-02-29,16389.0,18886.0,16389.0,17660.0,7.76,February,2000,02
2000-03-31,17668.0,19047.0,16922.0,17820.0,0.91,March,2000,03
2000-04-30,17826.0,17826.0,14029.0,15538.0,-12.81,April,2000,04
2000-05-31,15596.0,15944.0,13556.0,14957.0,-3.74,May,2000,05
...,...,...,...,...,...,...,...,...
2022-08-31,103165.0,114375.0,101694.0,109523.0,6.16,August,2022,08
2022-09-30,109524.0,114392.0,106244.0,110037.0,0.47,September,2022,09
2022-10-31,110048.0,120752.0,110048.0,116037.0,5.45,October,2022,10


In [18]:
resultado_geral = pd.pivot_table(ibov_ohlc, values="Resultado_mes", index=["Mes_num"], columns=[], aggfunc=np.mean)
resultado_geral = resultado_geral.style.format('{:.3f}', na_rep="").applymap(lambda x: 'background-color : #81b585' if x>0 else 'background-color : #cf7a7a')
resultado_geral

,Resultado_mes
Mes_num,
01,0.808
02,1.170
03,0.190
04,1.507
05,-0.910
06,-0.785
07,1.692
08,1.140
09,-0.815


In [19]:
resultados_pontuais = pd.pivot_table(ibov_ohlc, values="Resultado_mes", index=["Ano",], columns=["Mes_num"], aggfunc=np.mean)
resultados_pontuais = resultados_pontuais.style.format('{:.3f}', na_rep="").applymap(lambda x: 'background-color : #81b585' if x>0 else 'background-color : #cf7a7a')
resultados_pontuais

Mes_num,01,02,03,04,05,06,07,08,09,10,11,12
Ano,,,,,,,,,,,,
2000,-4.120,7.760,0.910,-12.810,-3.740,11.840,-1.630,5.420,-8.180,-6.660,-10.630,14.840
2001,15.820,-10.080,-9.140,3.320,-1.800,-0.610,-5.540,-6.640,-17.170,6.850,13.790,5.000
2002,-6.310,10.310,-5.540,-1.280,-1.710,-13.390,-12.350,6.340,-16.940,17.920,3.350,7.220
2003,-2.900,-6.030,9.660,11.380,6.890,-3.350,4.620,11.800,5.520,12.310,12.250,10.170
2004,-1.730,-0.440,1.780,-11.450,-0.320,8.210,5.620,2.090,1.940,-0.830,9.010,4.250
2005,-7.040,15.560,-5.430,-6.640,1.460,-0.620,3.960,7.690,12.620,-4.400,5.710,4.820
2006,14.730,0.590,-1.700,6.350,-9.500,0.280,1.220,-2.280,0.600,7.720,6.800,6.060
2007,0.380,-1.680,4.360,6.880,6.770,4.060,-0.380,0.840,10.670,8.030,-3.540,1.400
2008,-6.880,6.720,-3.970,11.320,6.960,-10.430,-8.480,-6.430,-11.030,-24.800,-1.770,2.610


In [20]:
pvalor_normalidade = ibov_ohlc[['Mes_num','Resultado_mes']].groupby('Mes_num').agg(lambda x: stats.normaltest(x).pvalue)
pvalor_normalidade

,Resultado_mes
Mes_num,
01,0.137649
02,0.685920
03,0.000082
04,0.754882
05,0.640768
06,0.901192
07,0.534325
08,0.512283
09,0.494974


In [21]:
mar = ibov_ohlc[ibov_ohlc.Mes=='March']
stats.normaltest(mar[mar.Ano!=2020]['Resultado_mes'])

NormaltestResult(statistic=2.720223565943761, pvalue=0.25663208825262684)

In [22]:
out = ibov_ohlc[ibov_ohlc.Mes=='October']
stats.normaltest(out[out.Ano!=2008]['Resultado_mes'])

NormaltestResult(statistic=0.5791453964548765, pvalue=0.7485833702485929)

In [24]:
ibov_ohlc_sem_crises = ibov_ohlc[~ibov_ohlc.Ano.isin([2008,2020])]
ibov_ohlc_sem_crises

,Open,High,Low,Close,Resultado_mes,Mes,Ano,Mes_num
Date,,,,,,,,
2000-01-31,17098.0,18099.0,15350.0,16388.0,-4.12,January,2000,01
2000-02-29,16389.0,18886.0,16389.0,17660.0,7.76,February,2000,02
2000-03-31,17668.0,19047.0,16922.0,17820.0,0.91,March,2000,03
2000-04-30,17826.0,17826.0,14029.0,15538.0,-12.81,April,2000,04
2000-05-31,15596.0,15944.0,13556.0,14957.0,-3.74,May,2000,05
...,...,...,...,...,...,...,...,...
2022-08-31,103165.0,114375.0,101694.0,109523.0,6.16,August,2022,08
2022-09-30,109524.0,114392.0,106244.0,110037.0,0.47,September,2022,09
2022-10-31,110048.0,120752.0,110048.0,116037.0,5.45,October,2022,10


In [27]:
pvalor_normalidade_sem_crises = ibov_ohlc_sem_crises[['Mes_num','Resultado_mes']].groupby('Mes_num').agg(lambda x: stats.normaltest(x).pvalue)
pvalor_normalidade_sem_crises

,Resultado_mes
Mes_num,
01,0.111705
02,0.446556
03,0.313153
04,0.979249
05,0.759374
06,0.776575
07,0.459188
08,0.741196
09,0.364258


In [28]:
jan = ibov_ohlc_sem_crises.query("Mes in 'January'")['Resultado_mes']
fev = ibov_ohlc_sem_crises.query("Mes in 'February'")['Resultado_mes']
mar = ibov_ohlc_sem_crises.query("Mes in 'March'")['Resultado_mes']
abr = ibov_ohlc_sem_crises.query("Mes in 'April'")['Resultado_mes']
mai = ibov_ohlc_sem_crises.query("Mes in 'May'")['Resultado_mes']
jun = ibov_ohlc_sem_crises.query("Mes in 'June'")['Resultado_mes']
jul = ibov_ohlc_sem_crises.query("Mes in 'July'")['Resultado_mes']
ago = ibov_ohlc_sem_crises.query("Mes in 'August'")['Resultado_mes']
setem = ibov_ohlc_sem_crises.query("Mes in 'September'")['Resultado_mes'] # set já é uma palavra reservada no python
out = ibov_ohlc_sem_crises.query("Mes in 'October'")['Resultado_mes']
nov = ibov_ohlc_sem_crises.query("Mes in 'November'")['Resultado_mes']
dez = ibov_ohlc_sem_crises.query("Mes in 'December'")['Resultado_mes']

In [29]:
stats.levene(jan, fev, mar, abr, mai, jun, jul, ago, setem, out, nov, dez)

LeveneResult(statistic=0.6898841360552501, pvalue=0.7477267667720155)

In [32]:
anova_medidas_rep = pg.rm_anova(data=ibov_ohlc_sem_crises, dv='Resultado_mes', within='Mes', subject='Ano', detailed=True)
anova_medidas_rep

,Source,SS,DF,MS,F,p-unc,ng2,eps
0,Mes,528.274852,11,48.024987,1.142335,0.329546,0.048631,0.623794
1,Error,9249.039556,220,42.041089,NaN,NaN,NaN,NaN


In [33]:
dados_media = ibov_ohlc_sem_crises[['Mes_num','Resultado_mes']].groupby(['Mes_num']).mean()
dados_sem = ibov_ohlc_sem_crises[['Mes_num','Resultado_mes']].groupby(['Mes_num']).std()
dados_figura = pd.concat([dados_media, dados_sem], axis=1)
dados_figura.columns = ['Media_retornos', 'SEM_retornos']

fig = go.Figure()

fig.add_trace(go.Bar(name='Retornos', x=dados_figura.index, y=dados_figura.Media_retornos,
                     error_y=dict(type='data', array=dados_figura.SEM_retornos),
                     marker_color='orange', orientation='v'))

fig.update_layout(title_text='<b>Retornos mensais do IBOV</b> (média \xB1 desvio padrão) <br><b>desde 2000', xaxis_title= '<b>Mês',
                  yaxis_title= '<b>Retorno médio mensal (%)', template="none", margin=dict(l=75, r=20, t=75, b=75),
                  paper_bgcolor="#f7f8fa", font=dict(family="Arial", size=13, color="black"), width=800, height=400)

fig.show()

In [34]:
df_post_hoc = pg.pairwise_ttests(data=ibov_ohlc_sem_crises, dv='Resultado_mes', within='Mes', subject='Ano')
df_post_hoc

d:\dev\TCD\Estatistica_Mercado_Financeiro\estats_mercado\lib\site-packages\pingouin\pairwise.py:28: UserWarning:

pairwise_ttests is deprecated, use pairwise_tests instead.



,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,Mes,April,August,True,True,-0.507392,20.0,two-sided,0.617430,0.256,-0.164146
1,Mes,April,December,True,True,-1.000854,20.0,two-sided,0.328854,0.355,-0.333312
2,Mes,April,February,True,True,-0.307803,20.0,two-sided,0.761415,0.237,-0.106481
3,Mes,April,January,True,True,-0.300269,20.0,two-sided,0.767071,0.237,-0.086911
4,Mes,April,July,True,True,-0.608646,20.0,two-sided,0.549610,0.269,-0.180538
...,...,...,...,...,...,...,...,...,...,...,...
61,Mes,May,October,True,True,-2.604393,20.0,two-sided,0.016965,3.257,-0.828754
62,Mes,May,September,True,True,-0.745469,20.0,two-sided,0.464660,0.292,-0.209492
63,Mes,November,October,True,True,-1.466044,20.0,two-sided,0.158183,0.576,-0.391340
64,Mes,November,September,True,True,0.594666,20.0,two-sided,0.558735,0.267,0.181359


In [35]:
df_post_hoc[df_post_hoc['p-unc']<0.05]

,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
17,Mes,August,May,True,True,2.306572,20.0,two-sided,0.031910,1.95,0.568420
24,Mes,December,June,True,True,2.268368,20.0,two-sided,0.034533,1.83,0.607677
26,Mes,December,May,True,True,2.643939,20.0,two-sided,0.015570,3.496,0.758407
56,Mes,March,May,True,True,2.102906,20.0,two-sided,0.048336,1.402,0.562934
61,Mes,May,October,True,True,-2.604393,20.0,two-sided,0.016965,3.257,-0.828754


In [37]:
df_ANOVA = pg.rm_anova(data=ibov_ohlc_sem_crises.query("Mes in ['March', 'May', 'October', 'December']"), dv='Resultado_mes', within='Mes', subject='Ano', detailed=True)
df_ANOVA

,Source,SS,DF,MS,F,p-unc,ng2,eps
0,Mes,366.996613,3,122.332204,3.21625,0.029022,0.109497,0.924634
1,Error,2282.139562,60,38.035659,NaN,NaN,NaN,NaN
